The goal of this notebook is to identify possible ways of collecting this data.

# Available repositories

## mwparserfromhell

In [ ]:
import mwapi
session = mwapi.Session('https://en.wikipedia.org', user_agent='williamvoje@gmail.com')


In [ ]:
page_request_data = session.get(action='query', titles ='git', prop = 'revisions', rvprop='content')
page_request_data['query']['pages']['1771747']['revisions'][0]['contentmodel']

In [ ]:
possible_page_data = page_request_data['query']['pages']['1771747']['revisions'][0]['*']

In [ ]:
import mwparserfromhell

In [ ]:
parsed = mwparserfromhell.parse(res["query"]["pages"]['1771747']['revisions'][0]['*'])

In [ ]:
parsed.filter_wikilinks()

In [ ]:
parsed.filter_headings()

In [ ]:
parsed.get_sections()[0].filter_wikilinks()

In [ ]:
len(parsed.strip_code())

Looks like mwparserfromhell will get the job done for parsing the data of raw information

In [ ]:
page_request_data = session.get(action='query', titles ='git', prop = 'revisions', rvprop='content')
page_request_data['query']['pages']['1771747']['revisions'][0]['contentmodel']

In [ ]:
page_request_data = session.get(action='query', titles ='Git', prop = 'revisions', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max', rvstop='2015-07-01T11:03:28Z')

In [ ]:
next(iter(page_request_data['query']['pages'].values()))['revisions']

In [ ]:
%%timeit
session.get(action='query', titles ='git', prop = 'revisions', 
                    rvprop='ids|flags|timestamp|comment|user|userid|content',
                    rvlimit='max')

Looks like we have all of the tools we need to start collecting this data I'm going to test this on a small page


First let's build out the tools we need to extract the data

In [ ]:
test_page = session.get(action='query', titles ='The_Faint', prop = 'revisions', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max')

In [ ]:
test_page['query']['pages']

In [ ]:
import pandas as pd
import os

In [ ]:
# This assumes that you are only looking at single page

page_title = 'The_Faint'
path_to_data = '../data/20180618_Test_history_data/'

headings = ['revid', 'parentid', 'user', 'userid', 'timestamp', 'comment', 
            'character_count', 'external_link_count', 'heading_count', 'wikilink_count', 'wikilinks']

for rev_count, revision_set in enumerate(session.get(continuation=True, action='query', titles =page_title, prop = 'revisions', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max')):
    
    # Make dataframe for storing information
    
    data_frame = pd.DataFrame(columns=headings)
    

    for count, revision in enumerate(next(iter(revision_set['query']['pages'].values()))['revisions']):

        # Extract edit information 
        revid, parentid, user, userid, timestamp, comment = revision_information(revision)

        # Extract information on the website itself
        parsed = mwparserfromhell.parse(revision['*'])

        character_count, external_link_count, heading_count, wikilink_count, wikilinks = parsed_article_metrics(parsed)
        
        data_frame.loc[count] = [ revid, parentid, user, userid, timestamp, comment,
                                 character_count, external_link_count, heading_count, wikilink_count, wikilinks]
        
    # Dump the dataframe
    data_frame.to_csv(os.path.join(path_to_data, page_title + "_" + str(rev_count) + '.csv'))
        
#         print(character_count, external_link_count, heading_count, wikilink_count)

In [ ]:
data_frame['wikilinks'][0]

In [ ]:
def revision_information(revision_json):
    """ This will export 
    revid, parentid, user, userid, timestamp, comment
    """
    return [revision['revid'], revision['parentid'], revision['user'], revision['userid'],
           revision['timestamp'], revision['comment']]


In [ ]:
def parsed_article_metrics(parsed_article):
    """This should take a parsed article
    Presently I'm using a mwparserfromhell"""
    character_count = len(parsed_article.strip_code())
    external_link_count = len(parsed_article.filter_external_links())
    heading_count = len(parsed_article.filter_headings())
    wikilink_count = len(set([str(link) for link in parsed_article.filter_wikilinks()]))            
    
    return character_count, external_link_count, heading_count, wikilink_count, parsed.filter_wikilinks()
    
    

In [ ]:
revid, parentid, user, userid, annon, timestamp, comment = revision_information(revision)

In [ ]:
len(set([str(link) for link in parsed.filter_wikilinks()]))

In [ ]:
parsed.filter_wikilinks()

In [ ]:
link = links[0]

In [ ]:
parsed.filter_wikilinks()[0].ipynb_checkpoints/

In [ ]:
link.title

In [ ]:
# process to complete

# init the process with the webpage 
session = mwapi.Session('https://en.wikipedia.org', user_agent='williamvoje@gmail.com')

# iterate through page revision history
for page_revisions in 





In [ ]:
import pandas as pd
import os

In [ ]:
# Consolidating all csvs in directory 
path_to_data = '../data/20180618_Test_history_data/'


df = pd.concat(pd.read_csv(
                
                for f in os.listdir(path_to_data)))




os.path.join(path_to_data, f) for f in os.listdir(path_to_data)

In [ ]:
[os.path.join(path_to_data, f) for f in os.listdir(path_to_data)]

In [ ]:
for x in os.walk(path_to_data):
    print(x)

In [ ]:
test = os.listdir(path_to_data)

In [ ]:
test

In [ ]:
import glob

In [ ]:
[i for i in glob.iglob(path_to_data + '*.csv')]

In [ ]:
df = pd.concat(pd.read_csv(i) for i in glob.iglob(path_to_data + '*.csv'))

In [ ]:
df['wikilinks']